In [2]:
import numpy as np
import random
from sklearn import metrics
from sklearn.datasets import fetch_mldata
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
np.random.seed(1)

### 1. Obtain the MNIST dataset and normalize it for use with a classifier.

In [4]:
mnist = fetch_mldata('MNIST original')

In [5]:
x_train, x_test, y_train, y_test = \
    train_test_split(mnist.data/255., mnist.target, test_size=0.2, random_state=0)

### 2. Select two different machine learning classification models

### A: logistic regression

In [12]:
# Apply Logistic regression classifier
clf = LogisticRegression(solver = 'lbfgs', n_jobs=1)

### 3.A Train both models on the MNIST dataset and achieve a decent testing accuracy

In [13]:
clf.fit(x_train, y_train)

# Make a prediction
y_pred = clf.predict(x_test)

In [11]:
print "Mean accuracy: {0:0.2f}".format(clf.score(x_test, y_test))

Mean accuracy: 0.92


### B: Random forest

In [6]:
# Apply Random forest classifier
clf = RandomForestClassifier(n_estimators=50,n_jobs=1)

### 3.B Train both models on the MNIST dataset and achieve a decent testing accuracy

In [7]:
clf.fit(x_train, y_train)

# Make a prediction
y_pred = clf.predict(x_test)

In [8]:
print "Mean accuracy: {0:0.2f}".format(clf.score(x_test, y_test))

Mean accuracy: 0.97
